# Deterministic Uncertainty Sets - Basic example

This section explains the basics of how to use the package with a determinsitic uncertainty set, and gives a simple example.

In [1]:
import cvxpy as cp
import scipy as sc
from sklearn import datasets
import numpy as np
import numpy.random as npr
import numpy.testing as npt
import torch
import pandas as pd
import lropt
import matplotlib.pyplot as plt
RTOL = 1e-04
ATOL = 1e-04
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

### Formulating the uncertainty set

The first step is to formulate the uncertainty set. There are multiple options, with their associated input parameters. The uncertainty set can be formulated within the uncertain parameter, which takes in the size of the uncertainty and an uncertainty set as parameters.

Ellipsoidal uncertainty: $\{u \mid \| Au + b \|_p \leq \rho \}$

- $\rho$ : float, optional  
  * Ellipsoid scaling. Default 1.0. 
- $p$ : integer, optional  
  * Order of the norm. Default 2.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.


In [2]:
# Ellipsoidal uncertainty set example
m = 5
data = np.random.normal(0,1,size = (100,m))
ellip_u = lropt.UncertainParameter(m,
                                   uncertainty_set = lropt.Ellipsoidal(p = 2, 
                                                                       rho=2., b = np.mean(data, axis = 0)))

Mean Robust uncertainty: $\{ u = (v_1,\dots,v_K)  \mid  \sum_{k=1} ^K w_k \| A_k(v_k - \bar{d}_k) \|_p^{\text{power}} \le \rho \}$ 

- $K$: int
  * Number of clusters. Default 1.
- data : np.array
  * Data to be clustered. The $K$ cluster centers are denoted $\bar{d}_k$ for $k = 1, \dots,K$.
- $\rho$ : float, optional  
  * Ellipsoid scaling. Default 1.0. 
- $p$ : integer, optional  
  * Order of the norm. Default 2.
- power: integer, optional
  * Power of the norm. Default 1. 
- $A_k$ : np.array, optional
  * Scaling matrix for u, for each $k$. Default identity matrix.
- Train: boolean, optional
  * Whether or not $A_k$ should be trained using given data. Default True.


In [3]:
# size of uncertain parameter
m = 5
# Generate data
data = np.random.normal(0,1,size = (100,m))

mro_u = lropt.UncertainParameter(m,
                                 uncertainty_set = lropt.MRO(K = 1, rho=2., data = data, train = False))

Box uncertainty: $\{ u \mid\| Au + b\|_\infty \leq \rho \}$ 

- $\rho$ : float, optional  
  * Box scaling. Default 1.0.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.


In [4]:
# Box uncertainty set example
box_u = lropt.UncertainParameter(m, 
                                 uncertainty_set = lropt.Box(rho=2.))

Budget uncertainty: $\{u \mid \| A_1u + b_1 \|_\infty \leq \rho_1, \| A_2u + b_2 \|_1 \leq \rho_2\}$ 

- $\rho_1$ : float, optional  
  * Box scaling. Default 1.0. 
- $\rho_2$ : float, optional
  * 1-norm scaling. Default 1.0.
- $A_1$, $A_2$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b_1$, $b_2$ : np.array, optional
  * Relocation vector for u. Default None.


In [5]:
# Budget uncertainty set example
budget_u = lropt.UncertainParameter(m, 
                                    uncertainty_set = lropt.Budget(rho1=2., 
                                                                   rho2 = 1.))

Polyhedral uncertainty: $\{ u \mid Du \leq d\}$

- $D$ : np.array  
- $d$ : np.array  

In [6]:
# Polyhedral uncertainty set example
poly_u = lropt.UncertainParameter(m, 
                                  uncertainty_set = lropt.Polyhedral(D = np.ones((3,m)), d = np.array([1,2,3])))

###  Formulating the Robust Problem

We can now fomulate the Robust Problem, treating the uncertain parameter as a regular parameter.

#### Example 1: Affine transformed LP, ellipsoidal uncertainty and mean robust uncertainty.
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu+a)^Tx \leq  10,\\
\end{array}
$$
where $c$, $P$, and $a$ are constants, and $u$ is the uncertain parameter from the ellipsoidal set above, where $b$ is nonzero.

In [7]:
#restate the ellipsoidal set
ellip_u = lropt.UncertainParameter(m,
                                  uncertainty_set = lropt.Ellipsoidal(p = 2, 
                                                                      rho=2., b = -np.mean(data, axis = 0)))
n = 4

# formulate cvxpy variable
x_r = cp.Variable(4)

# formulate problem constants
P = 3. * np.eye(m)[:n, :]
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(c@x_r)

# formulate constraints
constraints = [(P@ellip_u +a)@ x_r <= 10]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)


LRO objective value:  -2.097998029531417 
LRO x:  [-0.88502512 -1.15485879 -0.47530895 -0.34174012]


We note that this is equivalent to using the MRO uncertainty set with $K = 1$ and power = 1.

In [8]:
#
mro_u = lropt.UncertainParameter(m,
                                 uncertainty_set = lropt.MRO(rho=2., K = 1, data = data, train = False))
n = 4

# formulate cvxpy variable
x_m = cp.Variable(4)

# formulate objective
objective = cp.Minimize(c@x_m)

# formulate constraints
constraints = [(P@mro_u +a)@ x_m <= 10]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("MRO objective value: ", prob_robust.objective.value, "\nMRO x: ", x_m.value)


MRO objective value:  -2.097998028531319 
MRO x:  [-0.88502789 -1.15485947 -0.47530453 -0.34173626]


We compare the above with its explicit reformulation and solution
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx - b^T(P^Tx) + \rho\|P^Tx\|_2 \leq  10,\\
\end{array}
$$

In [9]:
# formulate cvxpy variable
x = cp.Variable(4)

# formulate objective
objective = cp.Minimize(c@x)

# formulate constraints
constraints = [a@x + np.mean(data, axis = 0)@(P.T@x) + 2*cp.norm(P.T@x,2) <= 10]

# formulate problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()

print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -2.0979980273174244 
Cvxpy x:  [-0.88503269 -1.15486066 -0.47529689 -0.34172959]


We see that they provide the same solution.